In [97]:
import numpy as np

In [110]:
A = [0, 1]

P_wait = np.array([
    [0.97, 0.01, 0.005, 0.005, 0.0025, 0.0025, 0.002, 0.002, 0, 0.001],
    [0.015, 0.965, 0.005, 0.005, 0.0025, 0.0012, 0.0012, 0.0025, 0.0013, 0.0013],
    [0.005, 0.005, 0.96, 0.01, 0.005, 0.005, 0.0025, 0.0025, 0.0025, 0.0025],
    [0, 0.005, 0.005, 0.95, 0.01, 0.01, 0.005, 0.005, 0.005, 0.005],
    [0, 0, 0.005, 0.005, 0.94, 0.02, 0.01, 0.01, 0.004, 0.006],
    [0, 0, 0, 0.005, 0.005, 0.93, 0.02, 0.02, 0.013, 0.007],
    [0, 0, 0, 0, 0.005, 0.007, 0.93, 0.03, 0.02, 0.008],
    [0, 0, 0, 0, 0, 0.005, 0.001, 0.914, 0.04, 0.04],
    [0, 0, 0, 0, 0, 0, 0.01, 0.01, 0.9, 0.08],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
])

S = list(range(0,11))  # 10 is transplant, 9 is death

rT = dict(zip(list(range(0,11)), [100,96,90,84,76,68,56,40,25,0, 0]))

rW = dict(zip(list(range(0, 11)), [0.99,0.98,0.97,0.96,0.93,0.9,0.87,0.84,0.8,0, 0]))
lam = 0.9998
epsilon = 1e-4
m_ = 3

In [111]:
def P(j, s, a):

    if s == 10 and j == 10:  # Transplant Absorbing State
        p = 1
    elif a == 1 and j == 10:  # Transplant (from non-transplant state)
        p = 1
    elif a == 0 and j != 10 and s != 10:
        p = P_wait[s, j]
    else:
        p = 0

    return p

def r(s, a):
    if a == 0:
        rwd = rW[s]
    elif a == 1:
        rwd = rT[s]
    else:
        rwd = 0

    return rwd
span = lambda v_: max(v_) - min(v_)

In [144]:
# INITIALIZATION

v = np.zeros(len(S))
v_prime = np.array([max(r(s, a_) + (lam * sum(P(j, s, a_) * v[j] for j in S)) for a_ in A) for s in S])
a_prime = np.random.choice(A, size=len(S))
while span(v_prime - v) > (lam * epsilon) / (1 - lam):
    m = 1
    u = v
    while m <= m_:

        u_prime = np.array([r(s, a_prime[s]) + lam * sum(P(j, s, a_prime[s]) * u[j] for j in S) for s in S])
        u = u_prime
        m += 1

    v = u_prime

    for s in S:
        v_prime[s] = max(r(s, a_) + (lam * sum(P(j, s, a_) * v[j] for j in S)) for a_ in A)
        a_prime[s] = np.argmax([r(s, a_) + (lam * sum(P(j, s, a_)* v[j] for j in S)) for a_ in A])

# TERMINATION

d_ep = a_prime
v_ep = v_prime + ((lam * (1 - lam)) * min(v_prime - v))

print('d = ', d_ep)
print('v:')
for var in range(len(v)):
    print(var, v[var])


d =  [0 0 0 1 1 1 1 1 1 0 0]
v:
0 100.0
1 96.0
2 90.0
3 84.0
4 76.0
5 68.0
6 56.0
7 40.0
8 25.0
9 0.0
10 0.0
